In [62]:
from flask import Flask, request
from flask.json import jsonify
from pprint import pprint
import pandas as pd

from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import inspect

import pymongo

from pyproj import Proj, transform

In [63]:
# This function adds the correct header name based on whether it was census year or not & also adds new column 'Year'
def create_basic_df_for_that_year(df, year, is_census):
    header = ["Street Address","Full Name","Race","Tenure","Census Name","Census Race"]
    # If there is No census info available for that year then make it explicitely blank data - so that 
    # the information across all the years is consistent
    
    if(is_census == 0):
        df["Census Name"] = ""
        df["Census Race"] = ""
    
    df.columns = header
    df["Year"] = year
    return df

In [64]:
# Added this function for future usage converting X & Y in ft into Lat & Long
def to_latlon(x, y):
    proj_in = 'epsg:2278'
    proj_out = 'epsg:4326'
    in_proj = Proj(init=proj_in, preserve_units=True)
    out_proj = Proj(init=proj_out)
    return transform(in_proj, out_proj, x, y)

In [3]:
# Retrieve info from the SQLite db - KEPT THIS IDEA ON HOLD! RIGHT NOW ONLY GETTING INFO FROM CSV
# -------------------------------------------------------------------------------------------------------
# engine = create_engine("sqlite:///Resources/digitalhumanity.db", connect_args={'check_same_thread': False})
# session = Session(engine)

In [61]:
# Retrieve info of X & Y (State Plane coordinates from CSV file) - THIS IS ALSO KEPT ON HOLD
# AS WE DON'T KNOW THE MAPPING COLUMN BETWEEN CENSUS DATA & BASIC INFO DATA
# ------------------------------------------------------------------------------------------------
# filename = "./Resources/censusrace1900_TableToExcel.csv"
# df_censuses_data = pd.read_csv(filename)

# list(df_censuses_data)

In [65]:
# Retrieve basic info for years 1900 through 1920 from CSV file
filename = "./Resources/basic info w race column_TableToExcel.csv"
df_basic_data_allyrs = pd.read_csv(filename)

In [66]:
df_basic_info_1900 = df_basic_data_allyrs.loc[:,["StNumClean","name1900","race1900","tenure1900","censusname1900","censusrace1900"]]
df_basic_info_1900 = create_basic_df_for_that_year(df_basic_info_1900, 1900, 1)

df_basic_info_1902 = df_basic_data_allyrs.loc[:,["StNumClean","name1902","race1902","tenure1902"]]
df_basic_info_1902 = create_basic_df_for_that_year(df_basic_info_1902, 1902, 0)

df_basic_info_1903 = df_basic_data_allyrs.loc[:,["StNumClean","name1903","race1903","tenure1903"]]
df_basic_info_1903 = create_basic_df_for_that_year(df_basic_info_1903, 1903, 0)

df_basic_info_1905 = df_basic_data_allyrs.loc[:,["StNumClean","name1905","race1905","tenure1905"]]
df_basic_info_1905 = create_basic_df_for_that_year(df_basic_info_1905, 1905, 0)

df_basic_info_1907 = df_basic_data_allyrs.loc[:,["StNumClean","name1907","race1907","tenure1907"]]
df_basic_info_1907 = create_basic_df_for_that_year(df_basic_info_1907, 1907, 0)

df_basic_info_1908 = df_basic_data_allyrs.loc[:,["StNumClean","name1908","race1908","tenure1908"]]
df_basic_info_1908 = create_basic_df_for_that_year(df_basic_info_1908, 1908, 0)

df_basic_info_1910 = df_basic_data_allyrs.loc[:,["StNumClean","name1910","race1910","tenure1910","censusname1910","censusrace1910"]]
df_basic_info_1910 = create_basic_df_for_that_year(df_basic_info_1910, 1910, 1)

df_basic_info_1911 = df_basic_data_allyrs.loc[:,["StNumClean","name1911","race1911","tenure1911"]]
df_basic_info_1911 = create_basic_df_for_that_year(df_basic_info_1911, 1911, 0)

df_basic_info_1912 = df_basic_data_allyrs.loc[:,["StNumClean","name1912","race1912","tenure1912"]]
df_basic_info_1912 = create_basic_df_for_that_year(df_basic_info_1912, 1912, 0)

df_basic_info_1913 = df_basic_data_allyrs.loc[:,["StNumClean","name1913","race1913","tenure1913"]]
df_basic_info_1913 = create_basic_df_for_that_year(df_basic_info_1913, 1913, 0)

df_basic_info_1915 = df_basic_data_allyrs.loc[:,["StNumClean","name1915","race1915","tenure1915"]]
df_basic_info_1915 = create_basic_df_for_that_year(df_basic_info_1915, 1915, 0)

df_basic_info_1917 = df_basic_data_allyrs.loc[:,["StNumClean","name1917","race1917","tenure1917"]]
df_basic_info_1917 = create_basic_df_for_that_year(df_basic_info_1917, 1917, 0)

df_basic_info_1918 = df_basic_data_allyrs.loc[:,["StNumClean","name1918","race1918","tenure1918"]]
df_basic_info_1918 = create_basic_df_for_that_year(df_basic_info_1918, 1918, 0)

df_basic_info_1919 = df_basic_data_allyrs.loc[:,["StNumClean","name1919","race1919","tenure1919"]]
df_basic_info_1919 = create_basic_df_for_that_year(df_basic_info_1919, 1919, 0)

df_basic_info_1920 = df_basic_data_allyrs.loc[:,["StNumClean","name1920","race1920","tenure1920","censusname1920","censusrace1920"]]
df_basic_info_1920 = create_basic_df_for_that_year(df_basic_info_1920, 1920, 1)

df_basic_info_1919.head()

,Street Address,Full Name,Race,Tenure,Census Name,Census Race,Year
0,805 Arthur Street,NOT LISTED,0,0,,,1919
1,830 Arthur Street,Mrs Daisy Davis,2,1,,,1919
2,832 Arthur Street,NOT LISTED,0,0,,,1919
3,834 Arthur Street,NOT LISTED,0,0,,,1919
4,836 Arthur Street,Cora Frazier ©,1,1,,,1919


In [75]:
# Initialize PyMongo to work with MongoDBs & create our OWN db to store information
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Retrieve OR Create New database and collection
db_DH = client.digitalHumanity_db
censuses_year1900_collection = db_DH.censuses_1900
censuses_year1902_collection = db_DH.censuses_1902
censuses_year1903_collection = db_DH.censuses_1903
censuses_year1905_collection = db_DH.censuses_1905
censuses_year1907_collection = db_DH.censuses_1907
censuses_year1908_collection = db_DH.censuses_1908
censuses_year1910_collection = db_DH.censuses_1910
censuses_year1911_collection = db_DH.censuses_1911
censuses_year1912_collection = db_DH.censuses_1912
censuses_year1913_collection = db_DH.censuses_1913
censuses_year1915_collection = db_DH.censuses_1915
censuses_year1917_collection = db_DH.censuses_1917
censuses_year1918_collection = db_DH.censuses_1918
censuses_year1919_collection = db_DH.censuses_1919
censuses_year1920_collection = db_DH.censuses_1920

In [76]:
# insert dataframes into respective MongoDB collections
censuses_year1900_collection.insert_many(df_basic_info_1900.to_dict('records'))
censuses_year1902_collection.insert_many(df_basic_info_1902.to_dict('records'))
censuses_year1903_collection.insert_many(df_basic_info_1903.to_dict('records'))
censuses_year1905_collection.insert_many(df_basic_info_1905.to_dict('records'))
censuses_year1907_collection.insert_many(df_basic_info_1907.to_dict('records'))
censuses_year1908_collection.insert_many(df_basic_info_1908.to_dict('records'))
censuses_year1910_collection.insert_many(df_basic_info_1910.to_dict('records'))
censuses_year1911_collection.insert_many(df_basic_info_1911.to_dict('records'))
censuses_year1912_collection.insert_many(df_basic_info_1912.to_dict('records'))
censuses_year1913_collection.insert_many(df_basic_info_1913.to_dict('records'))
censuses_year1915_collection.insert_many(df_basic_info_1915.to_dict('records'))
censuses_year1917_collection.insert_many(df_basic_info_1917.to_dict('records'))
censuses_year1918_collection.insert_many(df_basic_info_1918.to_dict('records'))
censuses_year1919_collection.insert_many(df_basic_info_1919.to_dict('records'))
censuses_year1920_collection.insert_many(df_basic_info_1920.to_dict('records'))
